In [1]:
# 原版本的获取连接

start_urls = ['http://books.toscrape.com/',]

def parse(self, response):
    for book_url in response.css("article.product_pod > h3 > a ::attr(href)").extract():
        yield scrapy.Request(response.urljoin(book_url), callback=self.parse_book_page)
    # 提取下一页的链接了
    next_page = response.css("li.next > a ::attr(href)").extract_first()
    if next_page:
        yield scrapy.Request(response.urljoin(next_page), callback=self.parse)

In [2]:
# 基于 scrapy.linkextractors 的获取连接
from scrapy.linkextractors import LinkExtractor

start_urls = ['http://books.toscrape.com/',]

def parse(self, response):
    for book_url in response.css("article.product_pod > h3 > a ::attr(href)").extract():
        yield scrapy.Request(response.urljoin(book_url), callback=self.parse_book_page)
    # 提取下一页的链接
    le = LinkExtractor(restrict_css='ul.pager li.next')
    links = le.extract_links(response)
    if links:
        next_url = links[0].url
        yield scrapy.Request(response.urljoin(next_url), callback=self.parse)

In [2]:
# 创建数据集
htmlpath4_1 = '/home/font/Data/Scrapy/example/example4_1.html'
htmlpath4_2 = '/home/font/Data/Scrapy/example/example4_2.html'

In [60]:
from scrapy.http import HtmlResponse
html4_1 = open(htmlpath4_1).read()
html4_2 = open(htmlpath4_2).read()

print(html4_1)
print(html4_2)

response4_1 = HtmlResponse(url='http://example4_1.com',body=html4_1,encoding='utf8')
response4_2 = HtmlResponse(url='http://example4_2.com',body=html4_2,encoding='utf8')

<!DOCTYPE html>
<html>
<head>
	<title>example4_1</title>
</head>
<body>

<div id="top" class="div1" style="width: 1230px">
	<p>下面是是一些站内链接</p>
	<a class="internal" href="/image1.html">Name:Image 1<br/><img src="image1.jpg">div_1 a_1</a>
	<a class="internal" href="/image2.html">Name:Image 2<br/><img src="image2.jpg">div_1 a_2</a>
	<a class="internal" href="/image3.html">Name:Image 3<br/><img src="image3.jpg">div_1 a_3</a>
	<a class="/img-red" href="/image4.html">Name:Image 4<br/><img src="image4.jpg">div_1 a_4</a>
</div>

<div id="bottom" style="width: 1230px">
	<a class="internal" href="/image5.html">Name:Image 5<br/><img src="image4.jpg"></a>
	<a class="internal" href="/image6.html">Name:Image 6<br/><img src="image6.jpg"></a>
</div>

<div id="images-3" style="width: 1230px">
	<p>下面是是一些站外链接</p>
	<a href="http://blog.csdn.net/FontThrone">Name:Image 7<br/><img src="image7.jpg">CSDN</a>
	<a href="https://github.com/FontTian">Name:Image 8<br/><img src="image8.jpg">Github</a>
	<a href="http:

**首先我们调用LinkExtractor来提取全部链接,之后我将介绍LinkExtractor中的各个参数.这些参数可以方便快捷的实现我们的很多想法**

In [15]:
from scrapy.linkextractors import LinkExtractor
le = LinkExtractor()
links = le.extract_links(response=response4_1)
for link in links:
    print(type(link)," ---> ",link.url)

<class 'scrapy.link.Link'>  --->  http://example4_1.com/image1.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image2.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image3.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image4.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image5.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image6.html
<class 'scrapy.link.Link'>  --->  http://blog.csdn.net/FontThrone
<class 'scrapy.link.Link'>  --->  https://github.com/FontTian
<class 'scrapy.link.Link'>  --->  http://www.cnblogs.com/fonttian/


## LinkExtractor中的参数
 - allow
   接受一个正则表达式火一个正则表达式列表,提取绝对url与正则表达式匹配的链接,如果该参数位空(默认),则提取全部链接
 - **示例** 提取网页中所有以'/images'开头的链接


In [24]:
pattern_allow = '/image[0-9]+\.html$'
le_allow = LinkExtractor(allow=pattern_allow)
links_allow = le_allow.extract_links(response=response4_1)
for link in links_allow:
    print(type(link)," ---> ",link.url)

<class 'scrapy.link.Link'>  --->  http://example4_1.com/image1.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image2.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image3.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image4.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image5.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image6.html


 - deny
   接受一个正则表达式火一个正则表达式列表,与allow相反,排除绝对url与正则表达式匹配的链接.
 - **示例** 通过deny 来提取页面中所哟偶的站外链接

In [23]:
from urllib.parse import urlparse
pattern_deny = '^'+urlparse(response4_1.url).geturl()
print(pattern_deny)

^http://example4_1.com


In [27]:
le_deny = LinkExtractor(deny=pattern_deny)
links_deny = le_deny.extract_links(response=response4_1)
for link in links_deny:
    print(type(link)," ---> ",link.url)

<class 'scrapy.link.Link'>  --->  http://blog.csdn.net/FontThrone
<class 'scrapy.link.Link'>  --->  https://github.com/FontTian
<class 'scrapy.link.Link'>  --->  http://www.cnblogs.com/fonttian/


 - allow_domains
   接受一个域名或一个域名列表,提取到指定域的链接
 - **示例** 提取页面中所有到github以及csdn的链接

In [30]:
domains = ['github.com','csdn.net']
le_domains = LinkExtractor(allow_domains=domains)
links_domains = le_domains.extract_links(response4_1)
for link in links_domains:
    print(type(link)," ---> ",link.url)

<class 'scrapy.link.Link'>  --->  http://blog.csdn.net/FontThrone
<class 'scrapy.link.Link'>  --->  https://github.com/FontTian


 - deny_domains
   接受一个域名或一个域名列表,与allow_domains相反排除到指定域的链接
 - ** 示例 ** 提取页面中除了到github外所有的链接

In [32]:
le_deny_dommains = LinkExtractor(deny_domains='github.com')
links_deny_dommains = le_deny_dommains.extract_links(response4_1)
for link in links_deny_dommains:
    print(type(link)," ---> ",link.url)

<class 'scrapy.link.Link'>  --->  http://example4_1.com/image1.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image2.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image3.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image4.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image5.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image6.html
<class 'scrapy.link.Link'>  --->  http://blog.csdn.net/FontThrone
<class 'scrapy.link.Link'>  --->  http://www.cnblogs.com/fonttian/


 - restrict_xpaths 
     接受一个XPath表达式或者一个XPatn表达式列表,提取Xpath表达式选中区域下的链接
 - **示例** 提取页面中```<div id = "top">```元素下的链接
  - restrict_css
     接受一个CSS选择器或者一个CSS选择器列表,提取CSS选择器选中区域下的链接
 - **示例** 提取页面中```<div id = "bottom">```元素下的链接

In [36]:
print('restrict_xpaths:')

le_xpath = LinkExtractor(restrict_xpaths='//div[@id="top"]')
links_xpath = le_xpath.extract_links(response4_1)
for link in links_xpath:
    print(type(link)," ---> ",link.url)

    
print('\nrestrict_css:')
le_css = LinkExtractor(restrict_css='div#bottom')
links_css = le_css.extract_links(response4_1)
for link in links_css:
    print(type(link)," ---> ",link.url)

restrict_xpaths:
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image1.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image2.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image3.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image4.html

restrict_css:
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image5.html
<class 'scrapy.link.Link'>  --->  http://example4_1.com/image6.html


 - tags
   接受一个标签(字符串)或者一个标签列表,提取指定标签内的链,默认位['a'.'area']
 - attrs
   接收一个属性(字符串)或一个属性列表,提取指定属性内的链接,默认为['href']

In [39]:
le_tags_attrs = LinkExtractor(tags = 'script',attrs = 'src')
links_tags_attrs = le_tags_attrs.extract_links(response=response4_2)

for link in links_tags_attrs:
    print(type(link)," ---> ",link.url)

<class 'scrapy.link.Link'>  --->  http://example4_2.com/js/app1.js
<class 'scrapy.link.Link'>  --->  http://example4_2.com/js/app2.js


 - process_vlaue
   接收一个形如func(vlaue)的回调函数,如果传递了该参数,LinkExtractor将调用该回调函数对提取每一个链接(如 a 的href)进行处理,回调函数正常情况下应该返回一个字符串的处理结果,想要抛弃所处理的连接时,则返回None.
 - **示例** 提取页面中的含义JavaScript代码的链接

In [64]:
import re
def process(value):
    m  = re.search("javascript:goToPage\('(.*?)'",value)
    # 如果匹配,就提取其中url并返回,如果不则返回原值
    if m:
        value = m.group(1)
    return value

le_process_value = LinkExtractor(process_value=process)
links_process_value = le_process_value.extract_links(response4_2)
for link in links_process_value:
    print(type(link)," ---> ",link.url)

<class 'scrapy.link.Link'>  --->  http://example4_2.com/home.html
<class 'scrapy.link.Link'>  --->  http://example4_2.com/doc.html
<class 'scrapy.link.Link'>  --->  http://example4_2.com/example.html
